In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import time

model_version = "microsoft/trocr-base-printed"
processor = TrOCRProcessor.from_pretrained(model_version)
model = VisionEncoderDecoderModel.from_pretrained(model_version)

C:\Users\yassi\Desktop\OCR\librairies-test\librairiestest\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
image = Image.open("DOTL.jpg").convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
start_time = time.time()  
generated_ids = model.generate(pixel_values)
end_time = time.time()  
extract_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
processing_time = end_time - start_time
print('output: ',extract_text)

C:\Users\yassi\Desktop\OCR\librairies-test\librairiestest\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


output:  :


In [3]:
print(processing_time)

12.932964324951172


In [4]:
crp_image = Image.open("DOTL_crop.jpg")

In [5]:
pixel_values = processor(crp_image, return_tensors="pt").pixel_values
start_time = time.time()  # Démarrer le chronomètre
generated_ids = model.generate(pixel_values)
end_time = time.time()  # Arrêter le chronomètre
extract_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
processing_time1 = end_time - start_time
print('Output from cropped image:', extract_text)

Output from cropped image: ***


In [6]:
print(processing_time1)

3.033796548843384


In [7]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image, ImageDraw
import pytesseract

# Configuration de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\yassi\AppData\Local\Tesseract-OCR\tesseract.exe'

# Chargement du modèle TrOCR
model_version = "microsoft/trocr-base-printed"
processor = TrOCRProcessor.from_pretrained(model_version)
model = VisionEncoderDecoderModel.from_pretrained(model_version)

# Fonction pour dessiner les boîtes englobantes avec pytesseract
def draw_boxes(image_path, output_path):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    width, height = image.size
    
    # Obtenir les boîtes englobantes des caractères avec pytesseract
    boxes = pytesseract.image_to_boxes(image)
    for b in boxes.splitlines():
        b = b.split(' ')
        x1, y2, x2, y1 = int(b[1]), height - int(b[2]), int(b[3]), height - int(b[4])
        draw.rectangle([x1, y1, x2, y2], outline=(0, 255, 0), width=2)
    
    image.save(output_path)
    image.show()

# Analyser et extraire le texte avec TrOCR
image_path = "DOTL.jpg"
image = Image.open(image_path).convert("RGB")
pixel_values = processor(image, return_tensors="pt").pixel_values
start_time = time.time() 
generated_ids = model.generate(pixel_values)
extract_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#print('Output:', extract_text)

# Dessiner les boîtes englobantes sur l'image originale
draw_boxes(image_path, "DOTL_result_with_boxes.png")
end_time = time.time()
processing_time = end_time - start_time
# Analyser et extraire le texte à partir de l'image recadrée avec TrOCR
crp_image_path = "DOTL_crop.jpg"
crp_image = Image.open(crp_image_path).convert("RGB")
pixel_values = processor(crp_image, return_tensors="pt").pixel_values
start_time1 = time.time() 
generated_ids = model.generate(pixel_values)
extract_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#print('Output from cropped image:', extract_text)

# Dessiner les boîtes englobantes sur l'image recadrée
draw_boxes(crp_image_path, "DOTL_result_crop_with_boxes.png")
end_time1 = time.time()
processing_time1 = end_time1 - start_time1

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(processing_time)

7.230514049530029


In [9]:
print(processing_time1)

3.264554738998413
